In [ ]:
import pandas as pd

Coleta dos dados

In [ ]:
estados = pd.read_csv('estados_brasileiros.csv', sep =';', encoding = 'latin-1')
estados.head()

,id_estado,estado,sigla_estado,pais
0,1,Acre,ac,Brasil
1,2,Alagoas,al,Brasil
2,3,Amapá,ap,Brasil
3,4,Amazonas,am,Brasil
4,5,Bahia,ba,Brasil


In [ ]:
clientes = pd.read_excel('dados_clientes.xlsx')
clientes.head()

,id_cliente,peso,colesterol,genero,id_estado
0,1,102.0,111,Masculino,23
1,2,115.0,135,Masculino,7
2,3,115.0,136,Masculino,4
3,4,140.0,167,Feminino,24
4,5,130.0,158,Masculino,26


In [ ]:
idade = pd.read_csv('idade_clientes.csv', sep =';', encoding = 'latin-1')
idade.head()

Preparação dos dados

In [ ]:
clientes.info()

In [ ]:
clientes.describe()

In [ ]:
estados.info()

In [ ]:
idade.info()

In [ ]:
idade.describe()

In [ ]:
clientes.isna().sum() # Verifica valores ausentes/nulos

id_cliente    0
peso          5
colesterol    0
genero        0
id_estado     0
dtype: int64

In [ ]:
estados.isna().sum()

id_estado       0
estado          0
sigla_estado    0
pais            2
dtype: int64

In [ ]:
idade.isna().sum()

In [ ]:
clientes.loc[clientes['peso'].isnull()] # Mostra as linhas onde possui valores nulos

Eliminação de dados ausentes

In [ ]:
clientes.dropna(subset=['peso'], inplace = True) # inplace = True -> alterar os dados naquele dataframe

In [ ]:
clientes.isna().sum()

id_cliente       0
peso             0
colesterol       0
genero           0
id_estado        0
genero_number    0
dtype: int64

In [ ]:
clientes.count()

id_cliente    542
peso          542
colesterol    542
genero        542
id_estado     542
dtype: int64

In [ ]:
estados.dropna(subset=['pais'], inplace = True)

In [ ]:
estados.isna().sum()

id_estado       0
estado          0
sigla_estado    0
pais            0
dtype: int64

In [ ]:
estados.count()

id_estado       25
estado          25
sigla_estado    25
pais            25
dtype: int64

Alteração dos dados ausentes

In [ ]:
media = clientes.peso.mean()
mediana = clientes.peso.median()
media, mediana

In [ ]:
round(media, 2) # Arredondando a variável média

In [ ]:
clientes.peso.fillna(round(media, 2), inplace = True) # Preenche os valores ausentes pela média

In [ ]:
clientes.isna().sum()

Substituição da variável categórica

In [ ]:
print(estados.pais.value_counts())

Brasil    27
Name: pais, dtype: int64


In [ ]:
estados.pais.fillna('Brasil', inplace = True) # Trocar o registro vazio na coluna país e muda o valor para Brasil

In [ ]:
estados.isna().sum()

id_estado       0
estado          0
sigla_estado    0
pais            0
dtype: int64

Análise da distribuição de dados

In [ ]:
import matplotlib.pyplot as plt 

In [ ]:
clientes.hist(column = 'peso', figsize = (15, 7), bins = 30)
plt.show()

Criação do identificador de gênero

In [ ]:
from sklearn.preprocessing import LabelEncoder # Codifica o label

In [ ]:
LE = LabelEncoder()
clientes['genero_number'] = LE.fit_transform(clientes['genero'])

In [ ]:
clientes

In [ ]:
clientes.hist(column = 'genero_number', figsize = (15, 7), bins = 3)
plt.show()

Junção entre os dados coletados

In [ ]:
estados[:5]

In [ ]:
clientes[:5] # Para verificar coluna em comum (nesse caso seria o id_estado)

In [ ]:
clientes_estados = pd.merge(clientes, estados, on = 'id_estado') # Junta as tabelas onde tiver o id_estado
clientes_estados.sort_values('id_cliente') # Ordena pelo id_cliente

In [ ]:
base_completa = pd.merge(clientes_estados, idade, on = 'id_cliente').sort_values('id_cliente')

In [ ]:
base_completa

In [ ]:
base_completa.reset_index(drop = True) # Organiza a tabela por um novo índice

Análise de dados

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
def calcular_wcss(dados_cliente):
  wcss = []
  for k in range(1, 11):
    kmeans = KMeans(n_clusters = k, random_state=0)
    kmeans.fit(X = dados_cliente)
    wcss.append(kmeans.inertia_)
  return wcss

In [ ]:
dados_cliente = base_completa[['peso', 'colesterol']]
dados_cliente.head()

In [ ]:
wcss_clientes = calcular_wcss(dados_cliente)

In [ ]:
for i in range(len(wcss_clientes)):
  print(f'o cluster {i} possui valor de WCSS de: {wcss_clientes[i]}')

o cluster 0 possui valor de WCSS de: 1248240.6686746986
o cluster 1 possui valor de WCSS de: 298932.7377352369
o cluster 2 possui valor de WCSS de: 127842.60787249672
o cluster 3 possui valor de WCSS de: 73809.93553059456
o cluster 4 possui valor de WCSS de: 49068.8738718445
o cluster 5 possui valor de WCSS de: 34064.50002100837
o cluster 6 possui valor de WCSS de: 25079.92393550797
o cluster 7 possui valor de WCSS de: 20378.05126656164
o cluster 8 possui valor de WCSS de: 16003.657266380145
o cluster 9 possui valor de WCSS de: 13385.38391038145


Visualização do gráfico do cotovelo

In [ ]:
import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go

In [ ]:
grafico_wcss = px.line(x = range(1, 11), 
                       y = wcss_clientes)
fig = go.Figure(grafico_wcss)

fig.update_layout(title = 'Calculando WCSS',
                  xaxis_title = 'Numero de clusters',
                  yaxis_title = 'Valor do WCSS',
                  template = 'plotly_white')
fig.show()

In [ ]:
kmeans_clientes = KMeans(n_clusters=4, random_state=0)
base_completa['cluster'] = kmeans_clientes.fit_predict(dados_cliente)

In [ ]:
base_completa

Calcular os centróides de cada cluster

In [ ]:
centroides_clusters = kmeans_clientes.cluster_centers_
centroides_clusters # Primeira coluna referente a peso e a segunda referente a colesterol

array([[184.32394366, 218.98591549],
       [127.57024793, 154.20661157],
       [106.90697674, 119.57364341],
       [152.32075472, 186.03773585]])

In [ ]:
grafico = px.scatter(x = base_completa['peso'],
                     y = base_completa['colesterol'],
                     color = base_completa['cluster'])

grafico_centroide = px.scatter(x = centroides_clusters[:, 0], y = centroides_clusters[:, 1], size = [7, 7, 7, 7])
grafico_final_petala = go.Figure( data = grafico.data + grafico_centroide.data)
grafico_final_petala.show()

Alterando o gráfico para melhor visualização

In [ ]:
grafico = px.scatter(x = base_completa['colesterol'],
                     y = base_completa['peso'],
                     color = base_completa['cluster'])

fig = go.Figure(grafico)
fig.update_layout(title = 'Análise de clusters',
                  xaxis_title = 'Colesterol',
                  yaxis_title = 'Peso')
fig.show()

Criação de função para plotar gráficos

In [ ]:
def plotar_grafico(x, y, x_label, y_label):
  grafico = px.scatter( x = x,
                       y = y,
                       color = base_completa['cluster'])
  
  fig = go.Figure(grafico)
  fig.update_layout(title = 'Analise de Clusters',
                    xaxis_title = x_label,
                    yaxis_title = y_label)
  fig.show()

In [ ]:
plotar_grafico(base_completa['colesterol'], base_completa['peso'], 'colesterol', 'peso')

Segmentando os clientes baseando a análise dos clusters

In [ ]:
x = 'colesterol'
y = 'peso'

plotar_grafico(base_completa[x], base_completa[y], x, y)

In [ ]:
cluster_clientes = base_completa

In [ ]:
cluster_clientes.loc[cluster_clientes['cluster']==0, 'nome_cluster'] = 'Alto Risco'
cluster_clientes.loc[cluster_clientes['cluster']==2, 'nome_cluster'] = 'Baixo Risco'
cluster_clientes.loc[cluster_clientes['cluster']==1, 'nome_cluster'] = 'Moderado Baixo'
cluster_clientes.loc[cluster_clientes['cluster']==3, 'nome_cluster'] = 'Moderado Alto'

In [ ]:
cluster_clientes

Análise dos clusters criado

In [ ]:
cluster_clientes.groupby('nome_cluster')['estado'].describe()

In [ ]:
cluster_clientes.groupby('nome_cluster')['peso'].describe()

,count,mean,std,min,25%,50%,75%,max
nome_cluster,,,,,,,,
Alto Risco,142.0,184.323944,9.764278,167.0,176.25,184.0,191.0,203.0
Baixo Risco,129.0,106.906977,6.313975,95.0,102.00,107.0,112.0,118.0
Moderado Alto,106.0,152.320755,7.023155,140.0,147.25,152.0,157.0,169.0
Moderado Baixo,121.0,127.570248,6.971163,115.0,122.00,128.0,133.0,142.0


In [ ]:
cluster_clientes.groupby('nome_cluster')['idade'].describe()

,count,mean,std,min,25%,50%,75%,max
nome_cluster,,,,,,,,
Alto Risco,142.0,43.204225,15.208965,16.0,32.00,42.5,56.00,69.0
Baixo Risco,129.0,43.139535,15.428132,17.0,30.00,41.0,58.00,70.0
Moderado Alto,106.0,42.716981,15.355318,16.0,29.25,44.0,54.75,70.0
Moderado Baixo,121.0,42.347107,14.411633,17.0,30.00,41.0,54.00,70.0


In [ ]:
cluster_clientes.groupby(['nome_cluster', 'genero'])['colesterol'].describe()

count        mean        std    min     25%    50%  \
nome_cluster   genero                                                          
Alto Risco     Feminino    60.0  220.650000   6.456701  204.0  216.75  221.0   
               Masculino   82.0  217.768293   8.939162  204.0  211.00  219.0   
Baixo Risco    Feminino    58.0  122.448276  10.372319  102.0  113.00  124.5   
               Masculino   71.0  117.225352   9.677953  102.0  109.00  115.0   
Moderado Alto  Feminino    59.0  186.152542  10.182496  168.0  176.00  189.0   
               Masculino   47.0  185.893617   9.311596  172.0  178.00  184.0   
Moderado Baixo Feminino    63.0  156.571429   9.596226  138.0  149.00  156.0   
               Masculino   58.0  151.637931   9.389614  138.0  143.25  151.5   

                            75%    max  
nome_cluster   genero                   
Alto Risco     Feminino   225.0  235.0  
               Masculino  225.0  235.0  
Baixo Risco    Feminino   130.0  138.0  
               Masculino  126.0  136.0  
Moderado Alto  Feminino   194.5  201.0  
               Masculino  192.5  203.0  
Moderado Baixo Feminino   166.0  171.0  
               Masculino  158.0  168.0